In [1]:
import torch 
import torch.nn as nn

In [2]:
# Residual Block
class Block(nn.Module):
    def __init__(self, in_channers, out_channels, identity_downsample=None, stride=1) -> None:
        """The Residual Block class

        Args:
            in_channers (int): The input channer
            out_channels (int): The output channels
            identity_downsample (any, optional): The Identity Down Samppling. Defaults to None.
            stride (int, optional): The Stride. Defaults to 1.
        """
        super(Block, self).__init__()
        self.expansion = 4 # the number of output channels is 4 times the number of input channels
        self.conv1 = nn.Conv2d(in_channers, out_channels, kernel_size=1, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        
    def forward(self, x):
        identity = x
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)
        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)
        x += identity
        x = self.relu(x)
        return x

In [ ]:
class ResNet(nn.Module): # [3, 4, 6, 3]
    def __init__(self, block, layers, image_channels, num_classes) -> None:
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(
            image_channels, 
            64, 
            kernel_size=7,
            stride=2,
            padding=3
        )
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(
            kernel_size=3, 
            stride=2, 
            padding=1
        )
        
        self.layer1 = self._make_layer(block, layers[0], out_channels=64, stride=1)
        self.layer2 = self._make_layer(block, layers[1], out_channels=128, stride=2)
        self.layer3 = self._make_layer(block, layers[2], out_channels=256, stride=2)
        self.layer4 = self._make_layer(block, layers[3], out_channels=512, stride=2)
        
        # average pool
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # fully connected layer
        self.fc = nn.Linear(512*4, num_classes)
        
    def forward(self, x):
        """
            The forward function

            Args:
                x (torch.Tensor): The input tensor

            Returns:
                torch.Tensor: The output tensor
        """
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        
        return x
        
    def _make_layer(self, block, num_residual_blocks, out_channels, stride):
        identity_downsample = None
        layers = []
        
        if stride != 1 or self.in_channels != out_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels, 
                    out_channels*4, 
                    kernel_size=1, 
                    stride=stride
                ),
                nn.BatchNorm2d(out_channels*4)
            )
            
        # This changes the number of channels, so we need to update in_channels for next residual block
        # The identity downsample is used to match the dimensions of the input and output of the residual block
        layers.append(
            block(
                self.in_channels,
                out_channels,
                identity_downsample,
                stride
            )
        )
        # update in_channels for next residual block
        self.in_channels = out_channels * 4
        
        # For example for first resnet layer: 256 will be mapped to 64 as first layer, then rest of the layers will keep output channels same
        for _ in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, out_channels))
        
        return nn.Sequential(*layers)
    
def ResNet50(img_channels=3, num_classes=1000):
    return ResNet(Block, [3, 4, 6, 3], img_channels, num_classes)

def Resnet101(img_channels=3, num_classes=1000):
    return ResNet(Block, [3, 4, 23, 3], img_channels, num_classes)

def ResNet152(img_channels=3, num_classes=1000):
    return ResNet(Block, [3, 8, 36, 3], img_channels, num_classes)

def test():
    net = ResNet50()
    x = torch.randn(2, 3, 224, 224)
    y = net(x).to("cpu")
    print(y.shape)